# Imports

In [1]:
import pandas as pd
import json
import validate
from general import many_to_many, many_cols
import ingest
import clean

In [2]:
def correct_values(db, table_name, column, wrong_value, correct_value):
    db[table_name][column]=db[table_name][column].apply(lambda x: x if x!=wrong_value else correct_value)

# Pull Data into Pandas

In [3]:
data = ingest.main('inventory_correct_linking.xlsx')

In [4]:
excel_data = data['excel_data']
db = data['database']

# Cleaning

In [5]:
db = clean.remove_NaN_columns(db)

In [6]:
db = clean.strip_all_columns(db)

In [7]:
# change ignore_code field to be string
db['gcmd_phenomena']['ignore_code'] = db['gcmd_phenomena']['ignore_code'].apply(lambda x: str(x))

# Short Name Supplementation

In [8]:
# make unique shortname by combining the campaign name and the table sub short name

db['deployment']['short_name'] = db['deployment']['foreign-campaign-short_name']+'_'+db['deployment']['ignore_deployment_id']
db['iopse']['foreign-deployment-short_name']=db['iopse']['foreign-campaign-short_name']+'_'+db['iopse']['ignore_deployment']

db['collection_period'] = many_to_many(db, 'linking', 'table-instrument-short_name', keep_all=True)
db['collection_period']['short_name'] = db['collection_period']['foreign-campaign-short_name']+'_'+db['collection_period']['foreign-deployment-short_name']

# Find Matching Deployments for IOPSE

In [9]:
# original process before good shortname usage on the sheets


# db['iopse']['deployment_short_name'] = 'No Matches'

# for row in range(len(db['iopse'])):
#     iop_start = db['iopse'].iloc[row]['start_date']
#     iop_end = db['iopse'].iloc[row]['end_date']
#     iop_camp = db['iopse'].iloc[row]['foreign-campaign-short_name']
    
#     campaign_filter = db['deployment']['foreign-campaign-short_name'].apply(lambda short_name: short_name == iop_camp)
#     possible_campaigns = db['deployment'][campaign_filter]
    
#     start_filter = possible_campaigns['start_date'].apply(lambda dep_start: validate.vali_date(dep_start, iop_start))
#     end_filter = possible_campaigns['end_date'].apply(lambda dep_end: validate.vali_date(iop_end, dep_end))
    
#     matching_deployments = list(possible_campaigns[start_filter*end_filter]['short_name'])
#     if len(matching_deployments)>1:
#         print(f"error on {db['iopse'].iloc[row]}")
#     elif len(matching_deployments)==1:
#         matching_deployments = matching_deployments[0]
#     else:
#         matching_deployments = 'None Found'
#     db['iopse']['deployment_short_name'].iloc[row]=matching_deployments

In [10]:
# test for unexpected values in this column
assert set(db['iopse']['type']) == {'IOP', 'SE'}

In [11]:
# convert parent and short name to lower so they will match correctly
db['iopse']['short_name'] = db['iopse']['short_name'].apply(lambda x: x.lower())
db['iopse']['parent short_name'] = db['iopse']['parent short_name'].apply(lambda x: x.lower())

In [12]:
db['iop'] = db['iopse'][db['iopse']['type']=='IOP']
db['significant_event'] = db['iopse'][db['iopse']['type']=='SE']

# Many to Many Creation

In [13]:
# main_table_names = ['campaign', 'platform', 'instrument', 'deployment']

In [14]:
# for table in main_table_names:
#     print(table)
#     for column in [col for col in db[table].keys() if isinstance(col,str) and 'table' in col]:
#         name = column.split('-')[1]
#         new_table_name = f"{table}-to-{name}"
#         db[new_table_name]=many_to_many(db, table, column)
#         print(f'   {new_table_name} created')

# Campaign Filter

In [15]:
campaign_filter = [
    "ACES",
    "AirMOSS",
    "ARCTAS",
    "CARVE",
    "DC3",
    "GCPEx",
    "GOES-R PLT",
    "GRIP",
    "HS3",
    "OLYMPEX"
]

In [16]:
# campaigns
db['campaign'] = db['campaign'][db['campaign']['short_name'].apply(lambda x: x in campaign_filter)]
assert set(campaign_filter) == set(db['campaign']['short_name'])

In [17]:
# deployments
db['deployment']=db['deployment'][db['deployment']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [18]:
# collection periods
db['collection_period']=db['collection_period'][db['collection_period']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [19]:
# iop
db['iop']=db['iop'][db['iop']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [20]:
# sig events
db['significant_event']=db['significant_event'][db['significant_event']['foreign-campaign-short_name'].apply(lambda short: short in campaign_filter)]

In [21]:
# platforms
platform_filter = list(set(list(db['collection_period']['foreign-platform-short_name'])))
db['platform']=db['platform'][db['platform']['short_name'].apply(lambda short: short in platform_filter)]
print('\ncopy these platforms into a file for inventory folks\n')
[print(thing) for thing in platform_filter]


copy these platforms into a file for inventory folks

UND Citation II
ER-2
ASO
G-V
B-200
P-3
C-23 Sherpa
DC-8
Citation
WB-57
Field_Site
Falcon
CV-580
G-III
GH
ALTUS II
ALAR


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [22]:
instrument_filter = list(set(list(db['collection_period']['instrument'])))
db['instrument']=db['instrument'][db['instrument']['short_name'].apply(lambda short: short in instrument_filter)]
print('\ncopy these instruments into a file for inventory folks\n')
[print(thing) for thing in instrument_filter]


copy these instruments into a file for inventory folks

2D-C/P
FEGS
AVOCET
Nephelometer
P-CIMS
PI-Neph
DV
CoSMIR
CPSD
CIMS
WS-CRDS
DOPS
CIP
HIWRAP
TDMA
AVIRIS
EFM
PILS
TD-LIF
AirMOSS
PSAP
AVAPS
DADS
WCN
CPI
BBR
CNC
LARGE
CSD CL
SP-2
DACOM
APR-2
RSP
CAPS
DIAL
EFCM
Nevzorov probe
EXRAD
CIT-CIMS
SSFR
SMPS
CPL
DIAL-HSRL
FGM
HIRAD
MSC
HD-SP2
AMS
CLH
King hot wire probe
ISAF
Information Not Available
Dropsonde
PCAP
SP2
VCSEL
DFGAS
PIP probe
CI-ITMS
CAR
OAP-2G-P
SR
OPC
SPEC
Accelerometer
ATHOS
HAMSR
DASH
CRS
NAWX radar
RICE
Spectrometer
WAS
FLIR
MMS
GT-CIMS
DAWN
AOP
PCASP
TOGA
ATSP
TDL
PALS
Gen-Chemistry
IR-CO2
PALMS
CAMS
CCN
Canisters
CPC
Slow Antenna
GC-MS
CDP
CCP
FPDS
ATLAS
PFP
Aerolaser
HSRL
LASE
HARP
LIP
DLH
SAGA
BAT probe
CAFS
UHSAS
Aethalometer
TE49C
FSSP
CSI
3V-CPI
Gen-AtmsState
FTS
HR-AMS
GCAS
Picarro
PTR-MS
HVPS-3
Gerdien Probe
S-HIS


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

# Many to Many Creation

In [23]:
main_table_names = ['campaign', 'platform', 'instrument', 'deployment']

In [24]:
for table in main_table_names:
    print(table)
    for column in [col for col in db[table].keys() if isinstance(col,str) and 'table' in col]:
        name = column.split('-')[1]
        new_table_name = f"{table}-to-{name}"
        db[new_table_name]=many_to_many(db, table, column)
        print(f'   {new_table_name} created')

campaign
   campaign-to-focus_area created
   campaign-to-season created
   campaign-to-platform_type created
   campaign-to-gcmd_phenomena created
   campaign-to-repository created
   campaign-to-partner_org created
   campaign-to-gcmd_project created
platform
   platform-to-gcmd_platform created
instrument
   instrument-to-gcmd_instrument created
   instrument-to-instrument_type created
   instrument-to-measurement_keywords created
   instrument-to-geophysical_concept created
   instrument-to-repository created
   instrument-to-measurement_region created
deployment
   deployment-to-geographical_region created


# Validation

### Short Name Duplicates

In [25]:
for table_name in db.keys():
    if table_name == 'collection_period':
        # this is being skipped because it has been broken out by instrument
        # and therefore it has duplicate short_names
        continue
        
    if 'short_name' in db[table_name].keys():
        print(table_name)
        duplicates = validate.find_duplicates(db, table_name, 'short_name')

        print(f'    {duplicates}')

platform_type
    []
home_base
    []
repository
    []
focus_area
    []
season
    []
instrument_type
    []
measurement_region
    []
geographical_region
    []
geophysical_concept
    []
campaign
    []
platform
    []
instrument
    ['atlas']
deployment
    []
iopse
    []
gcmd_instrument
    ['atlas', 'informationnotavailable', 'wcr', nan, 'epic', 'opc', 'gnssreceiver', 'cris', 'aa', 'iris', 'particlespectrometers', 'ssies', 'icecube', 'aps']
gcmd_platform
    ['kingair', 'informationnotavailable', 'goes10', 'goes11', 'goes12', 'goes13', 'goes14', 'goes15', 'goes16', 'goes1', 'goes2', 'goes3', 'goes4', 'goes5', 'goes6', 'goes7', 'goes8', 'goes9', 'environmentalmodeling']
gcmd_project
    ['afsisclimate', 'camp', 'informationnotavailable', 'iodp', 'landsat7', 'mcmurdopredatorprey', 'notapplicable']
partner_org
    []
iop
    []
significant_event
    []


### Foriegn Key Links

### Campaign

In [26]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_project', 
                                data_index='campaign', 
                                data_column='gcmd_project', 
                                foriegn_table='gcmd_project', 
                                foriegn_column='gcmd_uuid')
errors

,campaign,gcmd_project,suggestions


In [27]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-focus_area', 
                                data_index='campaign', 
                                data_column='focus_area', 
                                foriegn_table='focus_area', 
                                foriegn_column='short_name')
errors

,campaign,focus_area,suggestions


In [28]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-season', 
                                data_index='campaign', 
                                data_column='season', 
                                foriegn_table='season', 
                                foriegn_column='short_name')
errors

,campaign,season,suggestions


In [29]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-platform_type', 
                                data_index='campaign', 
                                data_column='platform_type', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

,campaign,platform_type,suggestions


In [30]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-gcmd_phenomena', 
                                data_index='campaign', 
                                data_column='gcmd_phenomena', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,campaign,gcmd_phenomena,suggestions


In [31]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-repository', 
                                data_index='campaign', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,campaign,repository,suggestions


In [32]:
errors = validate.foriegn_keys(db, 
                                data_table='campaign-to-partner_org', 
                                data_index='campaign', 
                                data_column='partner_org', 
                                foriegn_table='partner_org', 
                                foriegn_column='short_name')
errors

,campaign,partner_org,suggestions


In [33]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [34]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_keywords', 
                                data_index='instrument', 
                                data_column='measurement_keywords', 
                                foriegn_table='gcmd_phenomena', 
                                foriegn_column='ignore_code')
errors

,instrument,measurement_keywords,suggestions


In [37]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-gcmd_instrument', 
                                data_index='instrument', 
                                data_column='gcmd_instrument', 
                                foriegn_table='gcmd_instrument', 
                                foriegn_column='gcmd_uuid')
errors

,instrument,gcmd_instrument,suggestions
31,CIP,None,[]


In [38]:
correct_values(
    db=db,
    table_name = 'instrument-to-gcmd_instrument',
    column = 'gcmd_instrument',
    wrong_value = 'NID',
    correct_value = '6238f3e2-9a87-4e32-b866-c4a637094b51')
correct_values(
    db=db,
    table_name = 'instrument-to-gcmd_instrument',
    column = 'gcmd_instrument',
    wrong_value = '92f99316-b581-4adb-9980-aeb6bed64eee',
    correct_value = 'None')
correct_values(
    db=db,
    table_name = 'instrument-to-gcmd_instrument',
    column = 'gcmd_instrument',
    wrong_value = '6238fe2-9a87-4e32-b866-c4a637094b51',
    correct_value = '6238f3e2-9a87-4e32-b866-c4a637094b51')
correct_values(
    db=db,
    table_name = 'instrument-to-gcmd_instrument',
    column = 'gcmd_instrument',
    wrong_value = 'None',
    correct_value = '6238f3e2-9a87-4e32-b866-c4a637094b51')

In [41]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-instrument_type', 
                                data_index='instrument', 
                                data_column='instrument_type', 
                                foriegn_table='instrument_type', 
                                foriegn_column='short_name')
errors

,instrument,instrument_type,suggestions


In [40]:
correct_values(
    db=db,
    table_name = 'instrument-to-instrument_type',
    column = 'instrument_type',
    wrong_value = 'Passive - Remote Sensing',
    correct_value = 'Remote - Passive')
correct_values(
    db=db,
    table_name = 'instrument-to-instrument_type',
    column = 'instrument_type',
    wrong_value = 'Earth Remote Sensing - Active Remote Sensing',
    correct_value = 'Remote - Active')

In [46]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-geophysical_concept', 
                                data_index='instrument', 
                                data_column='geophysical_concept', 
                                foriegn_table='geophysical_concept', 
                                foriegn_column='short_name')
errors

,instrument,geophysical_concept,suggestions


In [45]:
correct_values(
    db=db,
    table_name = 'instrument-to-geophysical_concept',
    column = 'geophysical_concept',
    wrong_value = 'Precipitation',
    correct_value = 'Precipitation (Precip)')
correct_values(
    db=db,
    table_name = 'instrument-to-geophysical_concept',
    column = 'geophysical_concept',
    wrong_value = 'Terrestrial Hydrology',
    correct_value = 'Terrestrial Hydrology (TerrHydrol)')
correct_values(
    db=db,
    table_name = 'instrument-to-geophysical_concept',
    column = 'geophysical_concept',
    wrong_value = 'Atmospheric Chemicals & Trace Gasses',
    correct_value = 'Atmospheric Chemicals & Trace Gases')
correct_values(
    db=db,
    table_name = 'instrument-to-geophysical_concept',
    column = 'geophysical_concept',
    wrong_value = 'Cloud Properties',
    correct_value = 'Clouds-Properties & Processes & Dynamics	')
correct_values(
    db=db,
    table_name = 'instrument-to-geophysical_concept',
    column = 'geophysical_concept',
    wrong_value = 'Processes & Dynamics',
    correct_value = 'Boundary Layer Processes')

In [49]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-repository', 
                                data_index='instrument', 
                                data_column='repository', 
                                foriegn_table='repository', 
                                foriegn_column='short_name')
errors

,instrument,repository,suggestions


In [48]:
correct_values(
    db=db,
    table_name = 'instrument-to-repository',
    column = 'repository',
    wrong_value = 'ORNL',
    correct_value = 'OB.DAAC')

In [52]:
errors = validate.foriegn_keys(db, 
                                data_table='instrument-to-measurement_region', 
                                data_index='instrument', 
                                data_column='measurement_region', 
                                foriegn_table='measurement_region', 
                                foriegn_column='short_name')
errors

,instrument,measurement_region,suggestions


In [51]:
correct_values(
    db=db,
    table_name = 'instrument-to-measurement_region',
    column = 'measurement_region',
    wrong_value = 'Troposphere',
    correct_value = 'mid-troposphere')
correct_values(
    db=db,
    table_name = 'instrument-to-measurement_region',
    column = 'measurement_region',
    wrong_value = 'troposphere',
    correct_value = 'mid-troposphere')
correct_values(
    db=db,
    table_name = 'instrument-to-measurement_region',
    column = 'measurement_region',
    wrong_value = 'subsurface',
    correct_value = 'subsurface - water')

In [78]:
db['instrument-to-measurement_region'][db['instrument-to-measurement_region']['measurement_region']=='subsurface - water']

,instrument,measurement_region
6,AirMOSS,subsurface - water


In [54]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-geographical_region', 
                                data_index='deployment', 
                                data_column='geographical_region', 
                                foriegn_table='geographical_region', 
                                foriegn_column='short_name')
errors

,deployment,geographical_region,suggestions


In [55]:
[key for key in db.keys() if 'deployment' in key]

['deployment', 'deployment-to-geographical_region']

In [56]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment-to-platform', 
                                data_index='deployment', 
                                data_column='platform', 
                                foriegn_table='platform', 
                                foriegn_column='short_name')
print('\n\ndo I really need to validate this?')
errors

KeyError: 'deployment-to-platform'

In [57]:
errors = validate.foriegn_keys(db, 
                                data_table='deployment', 
                                data_index='short_name', 
                                data_column='foreign-campaign-short_name', 
                                foriegn_table='campaign', 
                                foriegn_column='short_name')
print('\n\n TODO this better once you have the data')
errors



 TODO this better once you have the data


6,short_name,foreign-campaign-short_name,suggestions


In [58]:
errors = validate.foriegn_keys(db, 
                                data_table='platform', 
                                data_index='short_name', 
                                data_column='foreign-platform_type-short_name', 
                                foriegn_table='platform_type', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-platform_type-short_name,suggestions


In [59]:
errors = validate.foriegn_keys(db, 
                                data_table='platform-to-gcmd_platform', 
                                data_index='platform', 
                                data_column='gcmd_platform', 
                                foriegn_table='gcmd_platform', 
                                foriegn_column='gcmd_uuid')
errors

,platform,gcmd_platform,suggestions


In [62]:
errors = validate.foriegn_keys(db, 
                                data_table='iop', 
                                data_index='short_name', 
                                data_column='foreign-deployment-short_name', 
                                foriegn_table='deployment', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-deployment-short_name,suggestions


In [63]:
correct_values(
    db=db,
    table_name = 'iop',
    column = 'foreign-deployment-short_name',
    wrong_value = 'AirMOSS_deb_2014b',
    correct_value = 'AirMOSS_dep_2014b')

In [65]:
errors = validate.foriegn_keys(db, 
                                data_table='significant_event', 
                                data_index='short_name', 
                                data_column='foreign-deployment-short_name', 
                                foriegn_table='deployment', 
                                foriegn_column='short_name')
errors

2,short_name,foreign-deployment-short_name,suggestions


### Flight

In [68]:
# flight table, instruments

errors = validate.foriegn_keys(db, 
                               data_table='collection_period', 
                              data_index='foreign-campaign-short_name', 
                              data_column='instrument', 
                              foriegn_table='instrument', 
                              foriegn_column='short_name')
errors

,foreign-campaign-short_name,instrument,suggestions


In [67]:
db['collection_period']=db['collection_period'][db['collection_period']['instrument']!='NAWX radar']

In [71]:
# Flight table, platforms

errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='foreign-platform-short_name', 
                      foriegn_table='platform', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,foreign-platform-short_name,suggestions


In [70]:
correct_values(
    db=db,
    table_name = 'collection_period',
    column = 'foreign-platform-short_name',
    wrong_value = 'UND Citation II',
    correct_value = 'Citation')
db['collection_period']=db['collection_period'][db['collection_period']['foreign-platform-short_name']!='Field_Site']

In [74]:
# I think to ignore this???

errors = validate.foriegn_keys(db, 
                      data_table='collection_period', 
                      data_index='foreign-campaign-short_name', 
                      data_column='short_name',#'foreign-deployment-short_name', 
                      foriegn_table='deployment', 
                      foriegn_column='short_name')
errors

,foreign-campaign-short_name,short_name,suggestions


In [73]:
correct_values(
    db=db,
    table_name = 'collection_period',
    column = 'short_name',
    wrong_value = 'OLYMPEX_dep_2016',
    correct_value = 'OLYMPEX_dep_2015')

db['collection_period']=db['collection_period'][db['collection_period']['short_name']!='Citation']

# Dates

In [ ]:
for table_name in db.keys():
    # find tables that have date fields
    if len(col_names := [col for col in db[table_name].columns if 'date' in col])>0:
        print(f"{table_name}\n    {', '.join(col_names)}")

In [ ]:
table_names = ['campaign', 'deployment', 'iopse'] 
for table_name in table_names:
    print(table_name)
    db[table_name]['valid_date']=False
    
    db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['start_date'], row['end_date']), axis=1)
        

In [ ]:
# this field no longer exists?

# table_names = ['instrument'] 
# for table_name in table_names:
    
#     db[table_name]['valid_date']=False
#     db[table_name]['valid_date'] = db[table_name].apply(lambda row: validate.vali_date(row['deployment_date'], row['decommision_date']), axis=1)
       

In [ ]:
db['campaign'][['short_name','start_date','end_date','valid_date']][db['campaign']['valid_date'].apply(lambda x: not(x))]

# IOPSE

In [ ]:
db['iop']

In [ ]:
# validate that all iops are unique, should return an empty list

db['iop']['short_name'][db['iop']['short_name'].duplicated()]

In [ ]:
# if sig event has matching iop, link them

# if sig event doesn't have matching iop, delete it so there will be no foriegn key

In [ ]:
# tag significant events that have IOPs

db['significant_event']['has_iop'] = db['significant_event']['iop_short_name'].apply(lambda short_name: short_name in list(db['iop']['iop_short_name']))


In [ ]:
# # check that all significant events with an iop have an iop

# s_i = set(db['iop']['iop_short_name'])
# s_s = set(has_iop['iop_short_name'])
# [s for s in s_s if s not in s_i]

In [ ]:
from datetime import datetime
# sig event start >= iop start

def start_val(sig_row, db):
    large_date = sig_row['start_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['start_date'].iloc[0]
    val = large_date >= small_date
    return val

def end_val(sig_row, db):
    large_date = sig_row['end_date']
    small_date = db['iop'][db['iop']['iop_short_name']==sig_row['iop_short_name']]['end_date'].iloc[0]
    val = large_date <= small_date
    return val

has_iop = db['significant_event'][db['significant_event']['has_iop']].copy()
val_iop_date_start = has_iop.apply(lambda row: not(start_val(row, db)), axis=1)
val_iop_date_end = has_iop.apply(lambda row: not(end_val(row, db)), axis=1)

In [ ]:
# display incorrect start dates
has_iop[val_iop_date_start]

In [ ]:
# display incorrect end dates
has_iop[val_iop_date_end]

## Sanity Check

In [ ]:
db['nasa_mission']

In [ ]:
db['collection_period']

In [ ]:
db['platform_type']

In [75]:
links = list(set(list(db['platform_type']['foreign-platform_type-short_name'])))
full = list(db['platform_type']['short_name'])
[link for link in links if link not in full]

['none']

### Pickle the Data

In [76]:
import pickle

In [77]:
pickle.dump(db, open('db_after_corrections','wb'))

In [ ]:
db['geographical_region']

In [ ]:
db['significant_event']